# Summary
Read the class info file for MiBIG entries, and report probability tables.

In [1]:
import os
import urllib
import glob
import json
import time
import numpy as np
import pandas as pd
from collections import OrderedDict

In [2]:
mibig_folder = "/mnt/scratch/louwe015/mibig_json_2.0/"

base_path = "/mnt/scratch/louwe015/NPLinker/classifying/mibig_classifications/"
out_base = os.path.split(base_path[:-1])[0]
mibig_classes = os.path.join(out_base, "All_MIBiG_compounds_with_AS_BGC_CF_NPC_classes.txt")
print("mibig_classes exists is", os.path.isfile(mibig_classes))

mibig_classes exists is True


## Reading file
Making dict of {chem_id: [[bgc_classes], [chem_classes]]}, where bgc_classes is [class, subclass, AS_class], and chem_classes is the CF/NPC classes in order of input file

In [3]:
classes_dict = {}
with open(mibig_classes) as inf:
    header = inf.readline()
    print(header)
    for line in inf:
        elems = line.strip().split("\t")
        chem_id = elems.pop(0)
        class_base = elems.pop(0).split(',')
        classes = [cls.partition(':')[0] for cls in class_base]
        sub_classes = [cls for cls in class_base if cls.split(":")[1]]
        as_classes = elems.pop(0).split(',')

        bgc_classes = [classes, sub_classes, as_classes]        
        chem_classes = [chem_cls.split('; ') for chem_cls in elems[2:]]
        classes_dict[chem_id] = [bgc_classes, chem_classes]

print(classes_dict[chem_id])  # example
print(classes_dict['BGC0000020_maytansine'])
classes_dict['BGC0000001_abyssomicin C']

compound_name	class:subclass	AS_classes	smiles	inchi_key	kingdom	superclass	class	subclass	direct_parent	class_results	superclass_results	pathway_results	isglycoside

[[['Other'], [], ['phosphonate']], [[''], [''], [''], [''], [''], [''], [''], [''], ['0']]]
[[['Polyketide'], [], ['T1PKS', 'NRPS-like']], [['Organic compounds'], ['Phenylpropanoids and polyketides'], ['Macrolactams'], [''], ['Macrolactams'], ['Ansa macrolides'], ['Macrolides'], ['Polyketides'], ['0']]]


[[['Polyketide'], ['Polyketide:Other'], ['T1PKS']],
 [['Organic compounds'],
  ['Organoheterocyclic compounds'],
  ['Oxanes'],
  [''],
  ['Oxanes'],
  ['Spirotetronate macrolides'],
  ['Macrolides'],
  ['Polyketides'],
  ['0']]]

In [4]:
# creating legend from the header
s_h = header.strip().split('\t')
legend_bgc = ['mibig_class']+s_h[1:3]
legend_chem = s_h[5:]
print(legend_bgc, legend_chem)

['mibig_class', 'class:subclass', 'AS_classes'] ['kingdom', 'superclass', 'class', 'subclass', 'direct_parent', 'class_results', 'superclass_results', 'pathway_results', 'isglycoside']


## Do comparisons between bgc and chem classes

In [5]:
from collections import defaultdict
def rec_dd():
    """Initialises a recurring defaultdict"""
    return defaultdict(rec_dd)

In [22]:
# aggregate pairwise class matrices for all compounds
result = rec_dd()
for chem_id, classes in classes_dict.items():
    bgc_classes, chem_classes = classes
    
    for i, bgc_cat in enumerate(legend_bgc):
        init_bgc_class = bgc_classes[i]
        
        #group pks-nrp hybrids
        hyb_count = len([1 for init_bgc_c in init_bgc_class if any([test in init_bgc_c.lower() for test in ['nrp', 'pks', 'polyketide']])])
        bgc_class = []
        if hyb_count >= 2:
            bgc_class.append("PKS-NRP-hybrid")
            for init_bgc_c in init_bgc_class:
                if any([test in init_bgc_c.lower() for test in ['nrp', 'pks', 'polyketide']]):
                    pass
                else:
                    bgc_class.append(init_bgc_c)
        else:
            bgc_class = init_bgc_class[:]
#         print(bgc_class, init_bgc_class)
        
        for j, chem_cat in enumerate(legend_chem):
            chem_class = chem_classes[j]
            
            for bgc_c in bgc_class:
                for chem_c in chem_class:
                    try:
                        result[bgc_cat][chem_cat][bgc_c][chem_c] += 1
                    except TypeError:
                        result[bgc_cat][chem_cat][bgc_c][chem_c] = 1

In [23]:
mibig_cls_npc_pathway = result['mibig_class']['pathway_results']  # MiBIG BGC class and NPC pathway
mibig_cls_npc_pathway

defaultdict(<function __main__.rec_dd()>,
            {'Polyketide': defaultdict(<function __main__.rec_dd()>,
                         {'Polyketides': 548,
                          '': 10,
                          'Shikimates and Phenylpropanoids': 20,
                          'Alkaloids': 72,
                          'Amino acids and Peptides': 24,
                          'Terpenoids': 35,
                          'Fatty acids': 14,
                          'Carbohydrates': 9}),
             'Other': defaultdict(<function __main__.rec_dd()>,
                         {'Polyketides': 35,
                          'Carbohydrates': 46,
                          'Amino acids and Peptides': 78,
                          'Alkaloids': 96,
                          'Shikimates and Phenylpropanoids': 21,
                          '': 9,
                          'Fatty acids': 19,
                          'Terpenoids': 5}),
             'Alkaloid': defaultdict(<function __main__.rec_d

In [24]:
df_mibig_cls_npc_pathway = pd.DataFrame.from_dict(mibig_cls_npc_pathway, dtype=int).fillna(0)
df_mibig_cls_npc_pathway

,Polyketide,Other,Alkaloid,Saccharide,Terpene,PKS-NRP-hybrid,NRP,RiPP
Polyketides,548,35,1.0,64.0,19.0,177,115.0,4.0
,10,9,0.0,7.0,0.0,7,8.0,2.0
Shikimates and Phenylpropanoids,20,21,0.0,3.0,1.0,6,9.0,0.0
Alkaloids,72,96,97.0,0.0,49.0,58,99.0,2.0
Amino acids and Peptides,24,78,12.0,9.0,2.0,187,320.0,113.0
Terpenoids,35,5,0.0,4.0,136.0,2,0.0,0.0
Fatty acids,14,19,0.0,0.0,0.0,12,2.0,0.0
Carbohydrates,9,46,0.0,60.0,1.0,1,8.0,0.0


In [25]:
df_mibig_cls_npc_pathway.style.background_gradient('Greens')

,Polyketide,Other,Alkaloid,Saccharide,Terpene,PKS-NRP-hybrid,NRP,RiPP
Polyketides,548,35,1.000000,64.000000,19.000000,177,115.000000,4.000000
,10,9,0.000000,7.000000,0.000000,7,8.000000,2.000000
Shikimates and Phenylpropanoids,20,21,0.000000,3.000000,1.000000,6,9.000000,0.000000
Alkaloids,72,96,97.000000,0.000000,49.000000,58,99.000000,2.000000
Amino acids and Peptides,24,78,12.000000,9.000000,2.000000,187,320.000000,113.000000
Terpenoids,35,5,0.000000,4.000000,136.000000,2,0.000000,0.000000
Fatty acids,14,19,0.000000,0.000000,0.000000,12,2.000000,0.000000
Carbohydrates,9,46,0.000000,60.000000,1.000000,1,8.000000,0.000000


In [26]:
s = df_mibig_cls_npc_pathway.style.bar(color=['#5fba7d'], axis=0, align='zero')
# s.style.bar(color=['#5fba7d'], axis=1, align='zero')
s

,Polyketide,Other,Alkaloid,Saccharide,Terpene,PKS-NRP-hybrid,NRP,RiPP
Polyketides,548,35,1.000000,64.000000,19.000000,177,115.000000,4.000000
,10,9,0.000000,7.000000,0.000000,7,8.000000,2.000000
Shikimates and Phenylpropanoids,20,21,0.000000,3.000000,1.000000,6,9.000000,0.000000
Alkaloids,72,96,97.000000,0.000000,49.000000,58,99.000000,2.000000
Amino acids and Peptides,24,78,12.000000,9.000000,2.000000,187,320.000000,113.000000
Terpenoids,35,5,0.000000,4.000000,136.000000,2,0.000000,0.000000
Fatty acids,14,19,0.000000,0.000000,0.000000,12,2.000000,0.000000
Carbohydrates,9,46,0.000000,60.000000,1.000000,1,8.000000,0.000000


## Visualisation

In [27]:
label_dict = OrderedDict()
label_dict.update({key:i for i, key in enumerate(mibig_cls_npc_pathway.keys())})
cur = len(label_dict)
for vals in mibig_cls_npc_pathway.values():
    for val in vals.keys():
        if val not in label_dict:
            label_dict[val] = cur
            cur += 1
label_vals = list(label_dict.keys())
label_dict, label_vals

(OrderedDict([('Polyketide', 0),
              ('Other', 1),
              ('Alkaloid', 2),
              ('Saccharide', 3),
              ('Terpene', 4),
              ('PKS-NRP-hybrid', 5),
              ('NRP', 6),
              ('RiPP', 7),
              ('Polyketides', 8),
              ('', 9),
              ('Shikimates and Phenylpropanoids', 10),
              ('Alkaloids', 11),
              ('Amino acids and Peptides', 12),
              ('Terpenoids', 13),
              ('Fatty acids', 14),
              ('Carbohydrates', 15)]),
 ['Polyketide',
  'Other',
  'Alkaloid',
  'Saccharide',
  'Terpene',
  'PKS-NRP-hybrid',
  'NRP',
  'RiPP',
  'Polyketides',
  '',
  'Shikimates and Phenylpropanoids',
  'Alkaloids',
  'Amino acids and Peptides',
  'Terpenoids',
  'Fatty acids',
  'Carbohydrates'])

In [28]:
source_vals = []
target_vals = []
value_vals = []

for bgc_c, vals in mibig_cls_npc_pathway.items():
    for chem_c, count in vals.items():
        source_vals.append(label_dict[bgc_c])
        target_vals.append(label_dict[chem_c])
        value_vals.append(count)

In [29]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label_vals
    ),
    link = dict(
      source = source_vals, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target_vals,
      value = value_vals
  ))])

fig.update_layout(title_text="MiBIG BGC classes vs NPClassifier pathways", font_size=10)
fig.show()

## Plotting function

In [31]:
from typing import Dict

def make_class_sankey_plot(class_dict: Dict[str, Dict[str, int]],
                           plot_title: str = "MiBIG BGC classes vs NPClassifier pathways"):
    label_dict = OrderedDict()
    label_dict.update({key:i for i, key in enumerate(class_dict.keys())})
    cur = len(label_dict)
    for vals in class_dict.values():
        for val in vals.keys():
            if val not in label_dict:
                label_dict[val] = cur
                cur += 1
    label_vals = list(label_dict.keys())

    source_vals = []
    target_vals = []
    value_vals = []

    for bgc_c, vals in class_dict.items():
        for chem_c, count in vals.items():
            source_vals.append(label_dict[bgc_c])
            target_vals.append(label_dict[chem_c])
            value_vals.append(count)

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = label_vals
        ),
        link = dict(
          source = source_vals, # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = target_vals,
          value = value_vals
      ))])

    fig.update_layout(title_text=plot_title, font_size=10)
    fig.show()
    return fig

In [39]:
print(result.keys())
result[list(result.keys())[0]].keys()

dict_keys(['mibig_class', 'class:subclass', 'AS_classes'])


dict_keys(['kingdom', 'superclass', 'class', 'subclass', 'direct_parent', 'class_results', 'superclass_results', 'pathway_results', 'isglycoside', 'is_glycoside'])

In [46]:
classes_result_dict = result['mibig_class']['isglycoside']
classes_result_dict

defaultdict(<function __main__.rec_dd()>,
            {'Polyketide': defaultdict(<function __main__.rec_dd()>,
                         {'0': 502, '1': 178}),
             'Other': defaultdict(<function __main__.rec_dd()>,
                         {'1': 48, '0': 246}),
             'Alkaloid': defaultdict(<function __main__.rec_dd()>,
                         {'0': 99, '1': 2}),
             'Saccharide': defaultdict(<function __main__.rec_dd()>,
                         {'1': 122, '0': 19}),
             'Terpene': defaultdict(<function __main__.rec_dd()>,
                         {'0': 157, '1': 21}),
             'PKS-NRP-hybrid': defaultdict(<function __main__.rec_dd()>,
                         {'0': 298, '1': 31}),
             'NRP': defaultdict(<function __main__.rec_dd()>,
                         {'0': 387, '1': 34}),
             'RiPP': defaultdict(<function __main__.rec_dd()>,
                         {'0': 113, '1': 2})})

In [47]:
classes_result_dict = result['mibig_class']['superclass']
make_class_sankey_plot(classes_result_dict, "MiBIG BGC classes vs ClassyFire superclass")